# Project Pre-Work

In [1]:
import acquire
import requests
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd

import re
import unicodedata
import nltk

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## Explore github repo:
1.  Need to identify programing language
2.  Need to scrape text, and save as corpus

<div class="alert alert-block alert-info"><b></b>

In [2]:
#<div class="alert alert-block alert-info"><b></b></div>

In [3]:
result = requests.get('https://github.com/microsoft/Web-Dev-For-Beginners')
result

<Response [200]>

In [4]:
print(result.headers)

{'server': 'GitHub.com', 'date': 'Wed, 18 Nov 2020 15:56:26 GMT', 'content-type': 'text/html; charset=utf-8', 'status': '200 OK', 'vary': 'X-PJAX, Accept-Encoding, Accept, X-Requested-With', 'etag': 'W/"52712de6badc094d57e0566d31b3d92e"', 'cache-control': 'max-age=0, private, must-revalidate', 'strict-transport-security': 'max-age=31536000; includeSubdomains; preload', 'x-frame-options': 'deny', 'x-content-type-options': 'nosniff', 'x-xss-protection': '1; mode=block', 'referrer-policy': 'no-referrer-when-downgrade', 'expect-ct': 'max-age=2592000, report-uri="https://api.github.com/_private/browser/errors"', 'content-security-policy': "default-src 'none'; base-uri 'self'; block-all-mixed-content; connect-src 'self' uploads.github.com www.githubstatus.com collector.githubapp.com api.github.com github-cloud.s3.amazonaws.com github-production-repository-file-5c1aeb.s3.amazonaws.com github-production-upload-manifest-file-7fdce7.s3.amazonaws.com github-production-user-asset-6210df.s3.amazona

In [5]:
src = result.content
src

b'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-v4AcZjAe20g8pbHzEi8vJXoGQRKB3EP48F4WKEztmlwB50jnHaghQvhUhiXO747gBQCylfFLWFAPmmyM/vpGkg==" rel="stylesheet" href="https://github.githubassets.com/assets/frameworks-bf801c66301edb483ca5b1f3122f2f25.css" />\n  <link crossorigin="anonymous" media="all" integrity="sha512-uP1AMyqEyOPRvRp69YW8vMoZjC+Zl+bvrF+7DpeyLNAlanwBk1G9TGFqr

In [6]:
soup =  BeautifulSoup(src, 'html.parser')
soup


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-bf801c66301edb483ca5b1f3122f2f25.css" integrity="sha512-v4AcZjAe20g8pbHzEi8vJXoGQRKB3EP48F4WKEztmlwB50jnHaghQvhUhiXO747gBQCylfFLWFAPmmyM/vpGkg==" media="all" rel="stylesheet">
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-b8fd40332a84c8e3d1bd1a7af585bcbc.css" integrity="sha512-uP1AMyqEyOPRvRp69YW8vMoZjC+Zl

In [7]:
# I want to find all the links and see them
li = soup.find_all('h1')
print(li)
print('\n')

[<h1 class="d-flex flex-wrap flex-items-center break-word f3 text-normal">
<svg aria-hidden="true" class="octicon octicon-repo-template text-gray mr-2" height="16" version="1.1" viewbox="0 0 16 16" width="16"><path d="M6 .75A.75.75 0 016.75 0h2.5a.75.75 0 010 1.5h-2.5A.75.75 0 016 .75zm5 0a.75.75 0 01.75-.75h1.5a.75.75 0 01.75.75v1.5a.75.75 0 01-1.5 0V1.5h-.75A.75.75 0 0111 .75zM4.992.662a.75.75 0 01-.636.848c-.436.063-.783.41-.846.846a.75.75 0 01-1.485-.212A2.501 2.501 0 014.144.025a.75.75 0 01.848.637zM2.75 4a.75.75 0 01.75.75v1.5a.75.75 0 01-1.5 0v-1.5A.75.75 0 012.75 4zm10.5 0a.75.75 0 01.75.75v1.5a.75.75 0 01-1.5 0v-1.5a.75.75 0 01.75-.75zM2.75 8a.75.75 0 01.75.75v.268A1.72 1.72 0 013.75 9h.5a.75.75 0 010 1.5h-.5a.25.25 0 00-.25.25v.75c0 .28.114.532.3.714a.75.75 0 01-1.05 1.072A2.495 2.495 0 012 11.5V8.75A.75.75 0 012.75 8zm10.5 0a.75.75 0 01.75.75v4.5a.75.75 0 01-.75.75h-2.5a.75.75 0 010-1.5h1.75v-2h-.75a.75.75 0 010-1.5h.75v-.25a.75.75 0 01.75-.75zM6 9.75A.75.75 0 016.75 9h2.5a.

## README.md Text

In [8]:
soup.select('h1', class_="Label Label--outline v-align-middle")[1].text

'Web Development for Beginners - A Curriculum'

In [9]:
soup.select('p', class_="markdown-body entry-content container-lg")[9].text

"Azure Cloud Advocates at Microsoft are pleased to offer a 12-week, 24-lesson curriculum all about JavaScript, CSS, and HTML basics. Each lesson includes pre- and post-lesson quizzes, written instructions to complete the lesson, a solution, an assignment and more. Our project-based pedagogy allows you to learn while building, a proven way for new skills to 'stick'."

In [10]:
soup.select('article', class_="markdown-body entry-content container-lg")[0].text

'Web Development for Beginners - A Curriculum\nAzure Cloud Advocates at Microsoft are pleased to offer a 12-week, 24-lesson curriculum all about JavaScript, CSS, and HTML basics. Each lesson includes pre- and post-lesson quizzes, written instructions to complete the lesson, a solution, an assignment and more. Our project-based pedagogy allows you to learn while building, a proven way for new skills to \'stick\'.\n\nTeachers, we have included some suggestions on how to use this curriculum. If you would like to create your own lessons, we have also included a lesson template\n\n\n\nClick the image above for a video about the project and the folks who created it!\n\nPedagogy\nWe have chosen two pedagogical tenets while building this curriculum: ensuring that it is project-based and that it includes frequent quizzes. By the end of this series, students will have built a typing game, a virtual terrarium, a \'green\' browser extension, a \'space invaders\' type game, and a business-type bank

## Program Language Text

In [11]:
soup.select('h2', class_="h4 mb-3")[12].text

'Languages'

In [12]:
soup.select('ul', class_="list-style-none")[18].text

'\n\n\n\nJavaScript\n81.9%\n\n\n\n\n\nHTML\n9.9%\n\n\n\n\n\nCSS\n8.2%\n\n\n'

In [129]:
def get_readme_articles(urls, cached=False):
    '''
    This function takes in a list of GitHub Repo urls and a parameter
    with default cached == False which scrapes the title, text, and language for each url, 
    creates a list of dictionary of features,converts list to df, and returns df.
    If cached == True, the function returns a df from a json file.
    '''
    if cached == True:
        df = pd.read_json('project_readme.json')
        
    # cached == False completes a fresh scrape for df     
    else:

        # Create an empty list to hold dictionaries
        text = []

        # Loop through each url in our list of urls
        for url in urls:

            # Make request and soup object using helper
            soup = make_soup(url)

            # Save the title of each repo in variable title
            title = soup.select('h1', class_="Label Label--outline v-align-middle")[0].text

            # Save the text in each repo to variable text
            content = soup.select('article', class_="markdown-body entry-content container-lg")[0].text
            
            # Save the language of each repo in variable language
            language = soup.select('li.d-inline:nth-child(1) > a:nth-child(1)')[0].text

            # Create a dictionary holding the title and content for each blog
            repo = {'title': title, 'content': content, 'language': language}

            # Add each dictionary to the articles list of dictionaries
            text.append(repo)
            
        # convert our list of dictionaries to a df
        df = pd.DataFrame(text)

        # Write df to a json file for faster access
        df.to_json('project_readme.json')
    
    return df

In [130]:
# Here cached == False, so the function will do a fresh scrape of the urls and write data to a json file.

urls = ['https://github.com/freeCodeCamp/freeCodeCamp',
        'https://github.com/996icu/996.ICU',
        'https://github.com/vuejs/vue',
        'https://github.com/EbookFoundation/free-programming-books',
        'https://github.com/facebook/react']

test = get_readme_articles(urls=urls, cached=False)
test

IndexError: list index out of range

In [18]:
test.columns

Index(['title', 'content', 'language'], dtype='object')

In [31]:
df = clean_data(test)
df

,title,content,language,text_cleaned,text_tokenized,text_lemmatized,text_filtered,words,doc_length
0,freecodecamp freecodecamp,\n\n\n\n\nfreeCodeCamp.org's open-source codeb...,basic javascript es6 regular expression debugg...,\n\n\n\n\nfreecodecamporgs opensource codebase...,freecodecamporgs opensource codebase and curri...,freecodecamporgs opensource codebase and curri...,freecodecamporgs opensource codebase curriculu...,"[freecodecamporgs, opensource, codebase, curri...",707
1,996icu 996icu,996.ICU\nPlease note that there exists NO othe...,update this list with evidence to help every w...,996icu\nplease note that there exists no other...,996icu\nplease note that there exists no other...,996icu please note that there exists no other ...,996icu please note exists official account app...,"[996icu, please, note, exists, official, accou...",456
2,vuejs vue,\n\n\n\n\n\n\n\n\n\n\nSupporting Vue.js\nVue.j...,,\n\n\n\n\n\n\n\n\n\n\nsupporting vuejs\nvuejs ...,supporting vuejs\nvuejs is an mitlicensed open...,supporting vuejs vuejs is an mitlicensed open ...,supporting vuejs vuejs mitlicensed open source...,"[supporting, vuejs, vuejs, mitlicensed, open, ...",256
3,ebookfoundation freeprogrammingbooks,This page is available as an easy-to-read webs...,all language,this page is available as an easytoread websit...,this page is available as an easytoread websit...,this page is available a an easytoread website...,page available easytoread website httpsebookfo...,"[page, available, easytoread, website, httpseb...",256
4,facebook react,React · \nReact is a JavaScript library for...,tutorial main concept advanced guide api refer...,react \nreact is a javascript library for ...,react \nreact is a javascript library for buil...,react react is a javascript library for buildi...,react react javascript library building user i...,"[react, react, javascript, library, building, ...",314


***

In [55]:
soup = make_soup('https://github.com/freeCodeCamp/freeCodeCamp')


In [133]:
soup.select('li.d-inline:nth-child(1) > a:nth-child(1)')[0].text

'\n\nJavaScript\n91.3%\n'

In [141]:
soup.select('html body.logged-in.env-production.page-responsive.intent-mouse div.application-main div main#js-repo-pjax-container div.container-xl.clearfix.new-discussion-timeline.px-3.px-md-4.px-lg-5 div.repository-content div.gutter-condensed.gutter-lg.flex-column.flex-md-row.d-flex div.flex-shrink-0.col-12.col-md-3 div.BorderGrid.BorderGrid--spacious div.BorderGrid-row div.BorderGrid-cell ul.list-style-none li.d-inline a.d-inline-flex.flex-items-center.flex-nowrap.link-gray.no-underline.text-small.mr-3')


[]

In [120]:
soup.select('div', class_="mb-2")[265].text

'\n\n\n\n\nlighthouserc.js\n\n\n\nfeat(tools): add LightHouse CI to the GitHub workflows (#39353)\n\n\n\nAug 6, 2020\n\n'

## Functions to Add to prepare file:

In [17]:
def make_soup(url):
    '''
    This helper function takes in a url and requests and parses HTML
    returning a soup object.
    '''
    response = get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [30]:
def clean_data(df):
    def basic_clean(text):
        text = (unicodedata.normalize('NFKD', text.lower())
                .encode('ascii', 'ignore') # ascii to reduce noise
                .decode('utf-8', 'ignore') # decode using utf-8
               )
        return re.sub(r"[^a-z0-9\s]", '', text)

    def tokenize(string):
        '''
        This function takes in a string and
        returns a tokenized string.
        '''
        # Create tokenizer.
        tokenizer = nltk.tokenize.ToktokTokenizer()

        # Use tokenizer
        string = tokenizer.tokenize(string, return_str=True)

        return string

    def lemmatize(string):
        '''
        This function takes in string for and
        returns a string with words lemmatized.
        '''
        # Create the lemmatizer.
        wnl = nltk.stem.WordNetLemmatizer()

        # Use the lemmatizer on each word in the list of words we created by using split.
        lemmas = [wnl.lemmatize(word) for word in string.split()]

        # Join our list of words into a string again and assign to a variable.
        string = ' '.join(lemmas)

        return string

    def remove_stopwords(string, extra_words=[], exclude_words=[]):
        '''
        This function takes in a string, optional extra_words and exclude_words parameters
        with default empty lists and returns a string.
        '''
        # Create stopword_list.
        stopword_list = stopwords.words('english')

        # Remove 'exclude_words' from stopword_list to keep these in my text.
        stopword_list = set(stopword_list) - set(exclude_words)
        # Add in 'extra_words' to stopword_list.
        stopword_list = stopword_list.union(set(extra_words))

        # Split words in string.
        words = string.split()

        # Create a list of words from my string with stopwords removed and assign to variable.
        filtered_words = [word for word in words if word not in stopword_list]

        # Join words in the list back into strings and assign to a variable.
        string_without_stopwords = ' '.join(filtered_words)

        return string_without_stopwords
    
    df['title'] = df.title.apply(basic_clean)
    df['title'] = df.title.apply(tokenize)
    df['title'] = df.title.apply(lemmatize)
    df['language'] = df.language.apply(basic_clean)
    df['language'] = df.language.apply(tokenize)
    df['language'] = df.language.apply(lemmatize)
    df['text_cleaned'] = df.content.apply(basic_clean)
    df['text_tokenized'] = df.text_cleaned.apply(tokenize)
    df['text_lemmatized'] = df.text_tokenized.apply(lemmatize)
    df['text_filtered'] = df.text_lemmatized.apply(remove_stopwords)
    # Add column with list of words
    words = [re.sub(r'([^a-z0-9\s]|\s.\s)', '', doc).split() for doc in df.text_filtered]
    df = pd.concat([df, pd.DataFrame({'words': words})], axis=1)
    # Adds colum with lenght of word list
    df['doc_length'] = [len(wordlist) for wordlist in df.words]
    return df.head()